# First of all...remember the imports!!!

Ohhhh...Python is so easy and nice 'cause there's always someone else doing the hard stuff...

![Image](../images/import.jpg)

In [1]:
import requests
import pandas as pd
import re
import json
import math

# ...and remenber to leave everything nice and tidy...

![Image](../images/spaghetti.jpg)


In [ ]:
API_TOKEN = 
USERNAME = 'Kristinawk' #USERNAME
BASE_URL = 'https://api.github.com/'
KEY = 'repos/'
OWNER = 'ih-datapt-mad/'
REPO = 'dataptmad0924_labs/' #LAB_REPOSITORY
SEARCH = 'search/issues?q=repo:'+OWNER+REPO+'+type:pr+state:{}'
PULLS = 'pulls?page={}&per_page=100&state={}'
COMMITS = 'pulls/{}/commits'
STATE = 'open'

In [6]:
field_list1 = ['number',
               'title',
               'state',
               'created_at',
               'updated_at',
               'closed_at',
               'html_url',
               'base.repo.full_name',
               'base.ref',
               'head.repo.full_name',
               'head.ref',
               'head.repo.pushed_at']

In [7]:
field_list2 = ['student_name',
               'number',
               'lab_name',
               'state',
               'lab_status',
               'created_at',
               'updated_at',
               'closed_at',
               'html_url',
               'base.repo.full_name',
               'base.ref',
               'head.repo.full_name',
               'head.ref',
               'head.repo.pushed_at']

In [8]:
field_sort1 = ['lab_status',
               'lab_name',
               'student_name']

In [9]:
field_name1 = ['Student Name',
               'PR Number',
               'Lab Name',
               'PR Status',
               'Lab Status',
               'PR Created at',
               'PR Updated at',
               'PR Closed at',
               'PR URL',
               'base repository',
               'base',
               'head repository',
               'compare',
               'Pushed at']

# Functions 

Here you may find all functions you need to create your Pipeline. Be sure you test them before you start to build your Pipeline.

![Image](../images/pieces.jpg)

### Auxiliary Functions

In [10]:
# Aux Function 1: You can get only 100 results per page so it is important to know the number of pages you'll need.

def pages(base_url, search, state, username, api_token):
    pages = requests.get(base_url + search.format(state), auth=(username,api_token)).json()['total_count']
    if STATE == 'open':
        pages = math.ceil(pages/100)
        return pages
    elif STATE == 'closed':
        pages = math.ceil(pages/100)
        return pages

In [11]:
# Auc Function 2: Check the committs in order to know which labs are ready to be reviewed.

def get_commits(base_url, key, owner, repo, commits, pull, username, api_token):
    r_commits = requests.get(base_url + key + owner + repo + commits.format(pull),
                             auth=(username, api_token)).json()
    df_commits = pd.json_normalize(r_commits)
    list_commits = list(df_commits['commit.message'])
    commit = list(set([commit if commit == 'lab-finished' else 'lab-started' for commit in list_commits]))
    if 'lab-finished' in commit:
        return 'lab-finished'
    else:
        return 'lab-started'

In [12]:
# Aux Function 3: But the students aren't careful with the naming...

def student_name(x):
    if ']' in x:
        x = x.split(']')
        x = x[1].replace('_', ' ').strip()
        len_x = len(x.split(' '))
        if len_x > 1:
            x = re.findall('\w[a-zA-Z áéíóúÁÉÍÓÚñÑ-]+', x)
            x = x[0].strip()
            return x
        else:
            x = 'No student name provided'
            return x
    else:
        x = 'Pull request is not properly named'
        return x

In [13]:
#Aux Function 4

def lab_name(x):
    if ']' in x:
        x = x.split(']')
        x = x[0] + ']'
        x = x.strip()
        lower_case = re.findall('[A-ZÁÉÍÓÚñÑ]+', x)
        if x[0] == '[' and x[-1] == ']' and ' ' not in x and len(lower_case) == 0:
            return x
        else:
            x = 'Lab format name is incorrect'
            return x
    else:
        x = 'Pull request is not properly named'
        return x

In [14]:
# Aux Function 5: ...or forget to push their work!!!

def time_parser(x):
    try:
        x = x.strip()
        x = re.findall('[0-9]+', x)
        x = ''.join(x)
        x = pd.to_datetime(x, format='%Y%m%d%H%M%S', errors='coerce')
        return x
    except:
        return 'Nothing pushed yet'

### Pipeline Functions

In [15]:
# Pipeline Function 1: And finally get the 'pull requests'.

def get_pulls(base_url, key, owner, repo, pulls, search, state, username, api_token, field_list):
    pulls_list = []
    max_pages = pages(base_url, search, state, username, api_token)
    for i in range(max_pages):
        r_pulls = requests.get(base_url + key + owner + repo + pulls.format(i+1, state),
                               auth=(username, api_token)).json()
        df_pulls = pd.json_normalize(r_pulls)
        pulls_list.append(df_pulls)
    df_pulls = pd.concat(pulls_list)
    df_pulls = df_pulls[field_list]
    return df_pulls

In [16]:
# Pipeline Function 2: Apply!!!!!!

def df_status(df_pulls, base_url, key, owner, repo, commits, username, api_token, field_list):
    df_pulls['student_name'] = df_pulls['title'].apply(student_name)
    df_pulls['lab_name'] = df_pulls['title'].apply(lab_name)
    df_pulls['created_at'] = df_pulls['created_at'].apply(time_parser)
    df_pulls['updated_at'] = df_pulls['updated_at'].apply(time_parser)
    df_pulls['head.repo.pushed_at'] = df_pulls['head.repo.pushed_at'].apply(time_parser)
    df_pulls['lab_status'] = df_pulls.apply(lambda col: get_commits(base_url,
                                                                    key,
                                                                    owner,
                                                                    repo,
                                                                    commits,
                                                                    col['number'],
                                                                    username,
                                                                    api_token), axis=1)
    df_status = df_pulls[field_list]
    return df_status

In [17]:
# Pipeline function 3: And there you have it!!!

def create_csv(df_status, field_sort, field_name):
    df_csv = df_status.sort_values(by=field_sort, ascending=False)
    df_csv.columns = field_name
    df_csv.to_csv('../data/labs_status.csv', index=False)
    return df_csv

In [19]:
# Damned Pipelines!!!

DF_PULLS = get_pulls(BASE_URL, KEY, OWNER, REPO, PULLS, SEARCH, STATE, USERNAME, API_TOKEN, field_list1)
DF_STATUS = df_status(DF_PULLS, BASE_URL, KEY, OWNER, REPO, COMMITS, USERNAME, API_TOKEN, field_list2)
DF_CSV = create_csv(DF_STATUS, field_sort1, field_name1)
DF_CSV

,Student Name,PR Number,Lab Name,PR Status,Lab Status,PR Created at,PR Updated at,PR Closed at,PR URL,base repository,base,head repository,compare,Pushed at
3,No student name provided,60,[web-scraping],open,lab-started,2024-10-26 11:35:14,2024-10-30 17:06:36,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,Kristinawk/dataptmad0924_labs,web-scraping,2024-10-30 18:47:37
6,Blanca de la Vega,56,[web-scraping],open,lab-started,2024-10-26 11:22:42,2024-10-26 12:50:36,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,blancadelavega/dataptmad0924_labs,web-scraping,2024-10-28 20:32:31
18,Blanca de la Vega,28,[string-operations],open,lab-started,2024-09-29 16:12:08,2024-09-29 18:44:51,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,blancadelavega/dataptmad0924_labs,string-operations,2024-10-28 20:32:31
9,Blanca de la Vega,53,[map-reduce-filter],open,lab-started,2024-10-20 18:42:39,2024-10-27 19:48:59,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,blancadelavega/dataptmad0924_labs,map-reduce-filter,2024-10-28 20:32:31
7,No student name provided,55,[lab-web-scraping],open,lab-started,2024-10-26 11:19:22,2024-11-02 10:11:09,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,chowira/dataptmad0924_labs,lab-web-scraping,2024-11-02 10:11:09
15,Paul Correa,37,[error-handling],open,lab-started,2024-10-08 13:08:45,2024-10-08 13:08:45,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,Paulcorreafl/dataptmad0924_labs,error-handling,2024-10-26 11:21:23
20,Blanca de la Vega,24,[error-handling],open,lab-started,2024-09-25 19:21:35,2024-09-25 19:41:50,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,blancadelavega/dataptmad0924_labs,error-handling,2024-10-28 20:32:31
24,jose luis gonzalez,8,[dicts-sets-tuples],open,lab-started,2024-09-19 00:20:57,2024-09-22 13:22:40,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,chowira/dataptmad0924_labs,dicts-sets-tuples,2024-11-02 10:11:09
21,Martin Gonzalez,15,[dicts-sets-tuples],open,lab-started,2024-09-21 13:06:50,2024-10-03 19:57:37,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,martingonzalezr001/dataptmad0924_labs,dicts-set-tuples,2024-11-02 10:10:04
12,No student name provided,49,[dataframe-calculations],open,lab-started,2024-10-14 19:11:14,2024-10-23 16:28:09,None,https://github.com/ih-datapt-mad/dataptmad0924...,ih-datapt-mad/dataptmad0924_labs,main,Kristinawk/dataptmad0924_labs,dataframe-calculations,2024-10-30 18:47:37
